In [14]:
using DelimitedFiles
using HDF5

A = readdlm("ww15mgh.grd", skipstart = 1)
A1 = permutedims(A); #transpose into column major order
a1 = vec(A1); #concatenate into a vector to remove unwanted string elements
a1_clean = filter(x -> !(x isa AbstractString), a1);
a_f32 = convert(Array{Float32, 1}, a1_clean) #cast to Float32
#we know that each sequence of 1441 elements corresponds to a constant-latitude
#set of values. after reshaping, each column is a constant-latitude set of
#values running over lon [-pi, pi]
A_F32 = reshape(a_f32, 1441, :); 
A_F32 = permutedims(A_F32); #transpose so that each row corresponds to a latitude value
#in the original text file latitude decreases from π/2 to -π/2, we need to flip
#it!
A_final = reverse(A_F32, dims = 1);

#write as big endian
open("ww15mgh_be.bin", "w") do f #opens the specified file, returns it as f, and applies to it the function in the block 
    write(f, hton.(A_final))
end

#write as little endian
open("ww15mgh_le.bin", "w") do f #opens the specified file, returns it as f, and applies to it the function in the block 
    write(f, htol.(A_final))
end

#write as HDF5
h5open("ww15mgh_hdf5.h5", "w") do f
    f["geoid_height"] = A_final
end



721×1441 Matrix{Float32}:
 -29.534  -29.534  -29.534  -29.534  …  -29.534  -29.534  -29.534  -29.534
 -29.547  -29.548  -29.549  -29.55      -29.545  -29.546  -29.546  -29.547
 -29.605  -29.607  -29.61   -29.612     -29.599  -29.601  -29.603  -29.605
 -28.964  -28.965  -28.966  -28.967     -28.961  -28.962  -28.963  -28.964
 -27.88   -27.878  -27.876  -27.874     -27.886  -27.884  -27.882  -27.88
 -27.088  -27.084  -27.08   -27.075  …  -27.102  -27.097  -27.093  -27.088
 -26.581  -26.576  -26.57   -26.564     -26.599  -26.593  -26.587  -26.581
 -25.836  -25.826  -25.816  -25.806     -25.867  -25.857  -25.847  -25.836
 -24.897  -24.877  -24.857  -24.838     -24.955  -24.936  -24.916  -24.897
 -24.246  -24.217  -24.188  -24.159     -24.335  -24.305  -24.276  -24.246
   ⋮                                 ⋱                               ⋮
  17.632   17.626   17.62    17.615      17.651   17.645   17.638   17.632
  17.042   17.035   17.029   17.022      17.064   17.057   17.05    17.042
  16

In [15]:
#read from big endian
A_tmp = Matrix{Float32}(undef, 721, 1441)
open("ww15mgh_be.bin", "r") do f
    read!(f, A_tmp)
end
A_retrieved = ntoh.(A_tmp)
@assert all(A_retrieved[end,:] .== Float32(13.606))
@assert all(A_retrieved[1,:] .== Float32(-29.534))

#read from little endian
A_tmp = Matrix{Float32}(undef, 721, 1441)
open("ww15mgh_le.bin", "r") do f
    read!(f, A_tmp)
end
A_retrieved = ltoh.(A_tmp)
@assert all(A_retrieved[end,:] .== Float32(13.606))
@assert all(A_retrieved[1,:] .== Float32(-29.534))

#read from HDF5
h5open("ww15mgh_hdf5.h5", "r") do f
    A_retrieved = f["geoid_height"] |> read
    A_retrieved[:,500] |> println
end
@assert all(A_retrieved[end,:] .== Float32(13.606))
@assert all(A_retrieved[1,:] .== Float32(-29.534))

SystemError: SystemError: opening file "ww15mgh_be.bin": No such file or directory

In [12]:
B = []